In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# Import some libraries
import numpy as np
import pandas as pd
import gc

/home/ramezani/anaconda/lib/python2.7/site-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [3]:
# Read the id_parcel and coordinates into the prop dataframe
path = "../../data/"
prop = pd.read_csv(path + 'renamed_properties_2016.csv', usecols = ["id_parcel", "latitude", "longitude"])
print "Size of the properties data frame: ", prop.shape

Size of the properties data frame:  (2985217, 3)


In [4]:
# Read the train data and merge with prop
train = pd.read_csv(path + 'renamed_train_2016.csv', usecols = ["id_parcel"])
train = pd.merge(train, prop, how='left', on='id_parcel')

In [5]:
# Drop the properties with missing coordinates
index = pd.isnull(prop["latitude"])
prop = prop[~index]
print prop.shape

(2973780, 3)


In [6]:
prop['latitude'] = prop['latitude'] / 1e6
prop['longitude'] = prop['longitude'] / 1e6

train['latitude'] = train['latitude'] / 1e6
train['longitude'] = train['longitude'] / 1e6

In [7]:
# Convert the dataframes to numpy arrays
X_prop = prop[['latitude','longitude']].as_matrix()
X_train = train[['latitude','longitude']].as_matrix()

In [8]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

In [15]:
def do_kmean(n_clusters, filename):
    '''
    takes a list of cluster sizes, and a filename. Creates the clusters and assigns each property 
    to a cluster, and then write the results into a csv file.
    '''
    for ii, nn in enumerate(n_clusters):
        print "fitting " + str(nn) + " clusters"         
        kmeans = MiniBatchKMeans(n_clusters=nn, random_state = 42, batch_size=1000).fit(X_train)
        name = "cluster" + str(ii)
        labels = kmeans.predict(X_train)
        prop[name] = kmeans.predict(X_prop)
        score = silhouette_score(X_train, labels, sample_size = 50000, random_state = 0)
        print "silhouette_score for " + str(nn) + "clusters is " + score
    
    print "wrtiting the csv file"
    prop.drop(['latitude','longitude'], axis=1).to_csv(path + filename, index=False)

In [ ]:
n_clusters = [80, 120, 290, 360, 470, 810]
do_kmean(n_clusters, 'n_clusters2.csv')